In [ ]:
# what is the use of @staticmethod 

**Inheritance** is a fundamental concept in OOP that allows a class (child/derived class) to inherit attributes and methods from another class (parent/base class). This promotes code reusability and establishes a hierarchical relationship between classes.

**Key Benefits:**

- Code reusability
- Logical hierarchy
- Extensibility
- Polymorphism support

In [2]:
class A:
    pass

class B(A):
    pass



#### Basic Single Inheritance

In [3]:
# Parent class
class Animal:
    def __init__(self, name, species):
        self.name = name
        self.species = species
    
    def make_sound(self):
        return "Some generic sound"
    
    def info(self):
        return f"{self.name} is a {self.species}"


# Child class inheriting from Animal
class Dog(Animal):
    def __init__(self, name, breed):
        # Calling parent constructor
        Animal.__init__(self, name, "Canine")
        self.breed = breed
    
    def make_sound(self):
        return "Woof! Woof!"
    
    def fetch(self):
        return f"{self.name} is fetching the ball!"


In [13]:
class Grandparent:
    pass

In [4]:
dog = Dog("Buddy", "Golden Retriever")

dog.species

'Canine'

In [5]:
dog.info()

'Buddy is a Canine'

In [6]:
# Using the classes
dog = Dog("Buddy", "Golden Retriever")
print(dog.info())           # Buddy is a Canine (inherited method)
print(dog.make_sound())     # Woof! Woof! (overridden method)
print(dog.fetch())          # Buddy is fetching the ball! (own method)

Buddy is a Canine
Woof! Woof!
Buddy is fetching the ball!


#### Multiple Inheritance

In [ ]:
class Flyer:

    def fly(self):
        return "Flying in the sky"

class Swimmer:
    def swim(self):
        return "Swimming in water"

class Duck(Animal, Flyer, Swimmer):
    def __init__(self, name):
        Animal.__init__(self, name, "Bird")

    
    
    # def make_sound(self):
    #     return "Quack! Quack!"

# Using multiple inheritance
duck = Duck("Donald")
print(duck.info())          # Donald is a Bird
print(duck.fly())           # Flying in the sky
print(duck.swim())          # Swimming in water
print(duck.make_sound())    # Quack! Quack!


Donald is a Bird
Flying in the sky
Swimming in water
Some generic sound


In [ ]:
# MRO

In [11]:
class Shape:
    def __init__(self, color):
        self.color = color
    def info(self):
        return f" This shape is for {self.color} color."

class Circle(Shape):
    def __init__(self, color, radius):
        Shape.__init__(self, color)
        self.radius = radius


c = Circle("blue", 6)
c.info()

' This shape is for blue color.'

type

## 1. Classes Are Objects: The Metaclass `type`

In Python, classes themselves are objects. They are created using a built-in metaclass called `type`.

> In Python, the fundamental principle is **everything is an object**.
> 

 > A **metaclass** is the 'class of a class'—it's what creates class objects. 

In [16]:
isinstance(Shape, type)

True


### Creation Flow

When you define a class using the `class` keyword:

```python
class Parent:
    pass
````

Python’s interpreter executes this definition, calling the `type` metaclass to construct the `Parent` object. This is equivalent to an internal call:


In [21]:
class Person:
    def greet():
        print("greet was called")

Person.__dict__['greet']



<function __main__.Person.greet()>

In [23]:
def func1():
    print("hi")

def func2():
    print("hello")

funcs = [func1, func2]

for f in funcs:
    f()


hi
hello


In [25]:
class Parent:
    pass

# Parent = type("Parent", (), {})

# "Parent" → the class name
# () → tuple of base classes (empty here)
# {} → dictionary containing class attributes and methods

### Class Attributes and References

Every class object therefore possesses:

- **A Namespace (`__dict__`):** A mapping (dictionary) of the class's attributes and methods.
- **Base Class Reference (`__bases__`):** A tuple of its immediate parent class(es).
- **MRO Reference (`__mro__`):** The computed Method Resolution Order (see Section 3).
- **Metaclass Reference (`__class__`):** A reference to the object that created it (i.e., `type`).


In [26]:
Parent.__bases__

(object,)


## 2. Defining and Linking a Child Class

When a subclass is created, its primary link to the parent is established via the `__bases__` attribute.

### Inheritance Definition

In [27]:
class Child(Parent):
    def child_method(self):
        print("I am a child.")

In [ ]:
Child = type(
    "Child", 
    (Parent,), # <--- The base class Parent is included here
    {'child_method': ...}
)

In [28]:
Child.__bases__

(__main__.Parent,)


### The Inheritance Link

The resulting class object `Child` holds a crucial reference:

```python
print(Child.__bases__)
# Output: (<class '__main__.Parent'>,)
```

**Inheritance is not an explicit copy operation.** The child class object simply stores a **reference** to its parent(s). When an attribute is requested on `Child`, if it's not found in `Child.__dict__`, Python follows these references to the base classes, initiating the **lookup process**.



## 3. Attribute Lookup and Method Resolution Order (MRO)

When an attribute or method is accessed on an instance, Python searches a predefined sequence of classes known as the **Method Resolution Order (MRO)**.

### MRO Mechanics

The MRO is a tuple of class objects, guaranteeing that a class is checked before its parents, and respecting the order of inheritance in multiple inheritance scenarios.

You can inspect the MRO using two equivalent methods:

In [30]:
Circle.mro()

[__main__.Circle, __main__.Shape, object]

In [31]:
class Parent:
    pass

class Child(Parent):
    pass

Child.__mro__
# or
Child.mro()



# (<class '__main__.Child'>, <class '__main__.Parent'>, <class 'object'>)

# Python searches in this order:

# 1. Child
# 1. Parent
# 2. object (the root of all classes)

# ------------------------------


class Grandparent: pass
class Parent(Grandparent): pass
class Child(Parent): pass

print(Child.mro())
# OR
print(Child.__mro__)

# Output for simple inheritance:
# (<class '__main__.Child'>, <class '__main__.Parent'>, <class '__main__.Grandparent'>, <class 'object'>)



[<class '__main__.Child'>, <class '__main__.Parent'>, <class '__main__.Grandparent'>, <class 'object'>]
(<class '__main__.Child'>, <class '__main__.Parent'>, <class '__main__.Grandparent'>, <class 'object'>)



### The `C3 Linearization` Algorithm

In more complex (especially multiple) inheritance, the MRO is calculated using the **C3 Linearization Algorithm**. Its core principle is:

> A class must precede all of its ancestors, and in the case of multiple inheritance, the order of the base classes specified in the definition must be preserved.

This ensures a deterministic and consistent lookup path, resolving potential conflicts that arise when multiple parents share a method name.


### The Lookup Process

1. Python starts the search in the **instance's dictionary** (`obj.__dict__`).
2. If not found, it iterates through the classes in the **MRO** (starting with the instance's class).    
3. For each class, it checks the class's **dictionary** (`Class.__dict__`).
4. If the attribute is found, the search stops and the attribute is returned.    
5. If the search reaches the end of the MRO (i.e., `object`) without success, an `AttributeError` is raised.



## 4. How `super()` Works Under the Hood

The `super()` function is the mechanism for delegated method calls, designed specifically to operate correctly within the MRO, particularly in complex multiple inheritance scenarios.

### Not a "Parent" Call

**Crucially, `super()` does not simply call the immediate parent class.** It is a _proxy object_ that provides the context for the next class in the MRO _after the class where `super()` is called_.

### Internal Steps of `super()`

When you call `super().method(...)`:

1. **MRO Determination:** Python identifies the current instance's MRO.
    
2. **Location:** It locates the class where the `super()` call is executed (e.g., `B` in the example below) within the MRO.
    
3. **Delegation:** It finds the **next class** in the MRO sequence (e.g., `A`).
    
4. **Invocation:** It calls the method (`.method()`) on that _next class_, **passing the original `self` instance along.**
    
> [!Note on syntax]
> 
> The most common modern usage is the "no-argument" form: `super().method()`.
> 
> Internally, Python 3 translates this into: `super(CurrentClass, self).method()`.




## 5. Method Binding and the Role of `self`

Instance methods are fundamentally different from regular functions because they are **bound** to an instance when they are accessed.

### The Descriptor Protocol

Methods are implemented using Python's **Descriptor Protocol**. A function object stored in a class's dictionary has a special method (`__get__`) that is executed when the function is retrieved via an instance.

When you call:

```python
obj = Child()
obj.method()
```

Python doesn't just execute the function; it first **binds** the function to the instance `obj`. This translation is approximately:

```python
# Internal translation
Class.method(obj) 
```

### `self` is the Instance

The `self` parameter in an instance method is simply the conventional name for the **reference to the instance object** that Python automatically passes as the _first argument_ during this binding process. Without the instance being explicitly passed, the method cannot access the instance's state (`obj.attribute`).

> That’s why every instance method must explicitly include self as its first parameter — it refers to the instance the method is called on.



## 6. The Complete Inheritance Execution Flow

### Example

In [ ]:
class A:
    def show(self):
        print("A: from class", self.__class__.__name__)
        super().

class B(A):
    def show(self):
        # MRO for B is (B, A, object)
        super().show() # Delegate to the next class (A)
        print("B: from class", self.__class__.__name__)

b = B()
b.show()

A: from class B
B: from class B



### Step-by-Step Execution

1. **`b.show()` Called:** Python looks up `show` in `b.__class__` (`B`).
2. **Binding:** The method `B.show` is found and bound to the instance `b`. The execution starts with the instance `b` as `self`.
3. **`super().show()`:**
    
    - `super()` determines the MRO is `(B, A, object)`.
        
    - The current class is `B`.
        
    - The **next class** is `A`.
        
    - It effectively calls `A.show(b)` (passing the original instance `b` as `self`).
        
4. **`A.show(b)` Executes:** It accesses `self.__class__.__name__` (which is still `B`) and prints the first line.
    
5. **Return to `B.show`:** Control returns to `B`'s method, which executes the next line.
    
6. **Second Print:** Prints the second line.
    

### Output:

```
A: from class B
B: from class B
```

This demonstrates how `super()` and the MRO ensure that methods from the entire inheritance chain can cooperate using a single, consistent instance (`b`).

---

## 7. Summary and Key Takeaways

### Core Concepts

- **Classes are Objects:** Defined by the `type` metaclass, possessing a namespace (`__dict__`) and base references (`__bases__`).
    
- **MRO Governs Lookup:** Attribute and method access is strictly governed by the **Method Resolution Order** (`__mro__`), which is computed using C3 Linearization.
    
- **`super()` is an MRO Proxy:** `super()` is a delegation mechanism that allows calls to the **next class in the MRO**, ensuring cooperative and consistent behavior, especially in multiple inheritance.
    
- **Binding Passes `self`:** Instance methods are automatically bound to the instance via the Descriptor Protocol, ensuring the instance is passed as the first argument (`self`).

```python
# Final review cheatsheet
print(Child.__bases__) # Immediate parent(s)
print(Child.mro())     # Full lookup chain
```

> [!tip] Best Practice
> 
> Always use the no-argument form of super() in your instance methods (e.g., super().__init__()). It is the most robust, readable, and future-proof way to manage cooperative inheritance.

---

Keep Learning!!!

In [33]:
object.__dict__

mappingproxy({'__new__': <function object.__new__(*args, **kwargs)>,
              '__repr__': <slot wrapper '__repr__' of 'object' objects>,
              '__hash__': <slot wrapper '__hash__' of 'object' objects>,
              '__str__': <slot wrapper '__str__' of 'object' objects>,
              '__getattribute__': <slot wrapper '__getattribute__' of 'object' objects>,
              '__setattr__': <slot wrapper '__setattr__' of 'object' objects>,
              '__delattr__': <slot wrapper '__delattr__' of 'object' objects>,
              '__lt__': <slot wrapper '__lt__' of 'object' objects>,
              '__le__': <slot wrapper '__le__' of 'object' objects>,
              '__eq__': <slot wrapper '__eq__' of 'object' objects>,
              '__ne__': <slot wrapper '__ne__' of 'object' objects>,
              '__gt__': <slot wrapper '__gt__' of 'object' objects>,
              '__ge__': <slot wrapper '__ge__' of 'object' objects>,
              '__init__': <slot wrapper '__init__' of